In [6]:
using Knet

┌ Info: Recompiling stale cache file /Users/kathi/.julia/compiled/v1.0/Knet/f4vSz.ji for Knet [1902f260-5fb4-5aff-8c31-6271790ab950]
└ @ Base loading.jl:1187


# Preprocessing
not implemented

# Initialization
implemented

In [ ]:
# initialization of weights (w/o batchnorm)
function init_model()
    w =  map(KnetArray, Any[ xavier(Float64,3,3,2,64),  zeros(Float64,1,1,64,1), # conv_1_1
            xavier(Float64,3,3,64,64), zeros(Float64,1,1,64,1), # conv_1_2
            xavier(Float64,3,3,64,128), zeros(Float64,1,1,128,1), # conv_2_1
            xavier(Float64,3,3,128,128), zeros(Float64,1,1,128,1) , # conv_2_2
            xavier(Float64,3,3,128,256), zeros(Float64,1,1,256,1), # conv_3_1 
            xavier(Float64,3,3,256,256), zeros(Float64,1,1,256,1), # conv_3_2
            xavier(Float64,3,3,256,512), zeros(Float64,1,1,512,1), # conv_4_1 
            xavier(Float64,3,3,512,512), zeros(Float64,1,1,512,1),  # conv_4_2
            xavier(Float64,3,3,768,256), zeros(Float64,1,1,256,1), #conv_5_1
            xavier(Float64,3,3,256,256), zeros(Float64,1,1,256,1), # conv_5_2
            xavier(Float64,3,3,384,128), zeros(Float64,1,1,128,1), # conv_6_1
            xavier(Float64,3,3,128,128), zeros(Float64,1,1,128,1), # conv_6_2
            xavier(Float64,3,3,192,64), zeros(Float64,1,1,64,1), # conv_7_1
            xavier(Float64,3,3,64,64), zeros(Float64,1,1,64,1), # conv_7_2
            xavier(Float64,1,1,64,2), zeros(Float64,1,1,2,1)]); # conv_7_3
    return w
end

In [ ]:
# initialization of weights (w/o batchnorm)
function init_model(depth, max_channels)
    # initialize w
    for layer in range(1,depth)
        
    w =  map(atype, Any[ xavier(Float64,3,3,2,64),  zeros(Float64,1,1,64,1), # conv_1_1
            xavier(Float64,3,3,64,64), zeros(Float64,1,1,64,1), # conv_1_2
            xavier(Float64,3,3,64,128), zeros(Float64,1,1,128,1), # conv_2_1
            xavier(Float64,3,3,128,128), zeros(Float64,1,1,128,1) , # conv_2_2
            xavier(Float64,3,3,128,256), zeros(Float64,1,1,256,1), # conv_3_1 
            xavier(Float64,3,3,256,256), zeros(Float64,1,1,256,1), # conv_3_2
            xavier(Float64,3,3,256,512), zeros(Float64,1,1,512,1), # conv_4_1 
            xavier(Float64,3,3,512,512), zeros(Float64,1,1,512,1),  # conv_4_2
            xavier(Float64,3,3,768,256), zeros(Float64,1,1,256,1), #conv_5_1
            xavier(Float64,3,3,256,256), zeros(Float64,1,1,256,1), # conv_5_2
            xavier(Float64,3,3,384,128), zeros(Float64,1,1,128,1), # conv_6_1
            xavier(Float64,3,3,128,128), zeros(Float64,1,1,128,1), # conv_6_2
            xavier(Float64,3,3,192,64), zeros(Float64,1,1,64,1), # conv_7_1
            xavier(Float64,3,3,64,64), zeros(Float64,1,1,64,1), # conv_7_2
            xavier(Float64,1,1,64,2), zeros(Float64,1,1,2,1)]); # conv_7_3
    return w
end

In [ ]:
# initialization of weights (w/ batchnorm)
function init_model_bn()
    # include m (moments for batchnorm)
    m = Any[bnmoments() for i = 1:n]# n = how often do we use BN?
    
    w =  map(atype, Any[ xavier(Float64,3,3,2,64),  zeros(Float64,1,1,64,1), # conv_1_1
            xavier(Float64,3,3,64,64), zeros(Float64,1,1,64,1), # conv_1_2
            xavier(Float64,3,3,64,128), zeros(Float64,1,1,128,1), # conv_2_1
            xavier(Float64,3,3,128,128), zeros(Float64,1,1,128,1) , # conv_2_2
            xavier(Float64,3,3,128,256), zeros(Float64,1,1,256,1), # conv_3_1 
            xavier(Float64,3,3,256,256), zeros(Float64,1,1,256,1), # conv_3_2
            xavier(Float64,3,3,256,512), zeros(Float64,1,1,512,1), # conv_4_1 
            xavier(Float64,3,3,512,512), zeros(Float64,1,1,512,1),  # conv_4_2
            xavier(Float64,3,3,768,256), zeros(Float64,1,1,256,1), #conv_5_1
            xavier(Float64,3,3,256,256), zeros(Float64,1,1,256,1), # conv_5_2
            xavier(Float64,3,3,384,128), zeros(Float64,1,1,128,1), # conv_6_1
            xavier(Float64,3,3,128,128), zeros(Float64,1,1,128,1), # conv_6_2
            xavier(Float64,3,3,192,64), zeros(Float64,1,1,64,1), # conv_7_1
            xavier(Float64,3,3,64,64), zeros(Float64,1,1,64,1), # conv_7_2
            xavier(Float64,1,1,64,2), zeros(Float64,1,1,2,1)]); # conv_7_3
    return w, m
    end

In [1]:
function init_model(n_input_channels=1, n_output_channels=2, depth=4, max_channels=512, kernel_size=3)
    w = Any[]
    bn_params = Any[]
    bn_moments = Any[bnmoments() for i = 1:14]
    # 1st layer
    # convolution
    push!(w,xavier(Float64,kernel_size,kernel_size,n_input_channels,Int64(max_channels/(2^(depth-1)))))
    # bias
    push!(w,zeros(Float64,1,1,Int64(max_channels/(2^(depth-1))),1))
    # batchnorm params
    push!(bn_params, bnparams(Float64,Int64(max_channels/(2^(depth-1)))))
    # convolution
    push!(w,xavier(Float64,kernel_size,kernel_size,Int64(max_channels/(2^(depth-1))),Int64(max_channels/(2^(depth-1)))))
    # bias
    push!(w,zeros(Float64,1,1,Int64(max_channels/(2^(depth-1))),1))
    push!(bn_params, bnparams(Float64,Int64(max_channels/(2^(depth-1)))))
    
    println("layer 1")
    println(size(w))
    
    # encoding arm: 2nd up to and including bottleneck
    for layer=2:depth
       push!(w,xavier(Float64,kernel_size,kernel_size,Int64(max_channels/(2^(depth-layer+1))),Int64(max_channels/(2^(depth-layer))))) 
        push!(w, zeros(Float64,1,1,Int64(max_channels/(2^(depth-layer))),1))
        push!(bn_params, bnparams(Float64,Int64(max_channels/(2^(depth-layer)))))
        push!(w,xavier(Float64,kernel_size,kernel_size,Int64(max_channels/(2^(depth-layer))),Int64(max_channels/(2^(depth-layer))))) 
        push!(w, zeros(Float64,1,1,Int64(max_channels/(2^(depth-layer))),1))
        push!(bn_params, bnparams(Float64,Int64(max_channels/(2^(depth-layer)))))
    end
    
    # decoding arm (except for 3rd convolution in the last layer)
    for layer=(depth+1):(2*depth-1)
        push!(w,xavier(Float64,kernel_size,kernel_size,Int64(1.5*max_channels/(2^(layer-depth-1))),Int64(max_channels/(2^(layer-depth))))) 
        push!(w, zeros(Float64,1,1,Int64(max_channels/(2^(layer-depth))),1))
        push!(bn_params, bnparams(Float64,Int64(max_channels/(2^(layer-depth)))))
        push!(w,xavier(Float64,kernel_size,kernel_size,Int64(max_channels/(2^(layer-depth))),Int64(max_channels/(2^(layer-depth))))) 
        push!(w, zeros(Float64,1,1,Int64(max_channels/(2^(layer-depth))),1))
        push!(bn_params, bnparams(Float64,Int64(max_channels/(2^(layer-depth)))))
    end
    
    # output convolution
    push!(w, xavier(Float64,kernel_size,kernel_size,Int64(max_channels/(2^(depth-1))),n_output_channels)) 
    push!(w, zeros(Float64,1,1,n_output_channels,1))
       
    w = map(KnetArray, w)
    bn_params = map(KnetArray, bn_params)
    
    return w, bn_moments, bn_params
end

init_model (generic function with 6 methods)

# model
IMPLEMENTED

In [1]:
function cat3d_(arrs...)
      H = size(first(arrs), 1)
      W = size(first(arrs), 2)
      C = sum(size(a, 3) for a in arrs)
      T = size(first(arrs), 4)

      @assert all(H == size(a, 1) for a in arrs)
      @assert all(W == size(a, 2) for a in arrs)
      @assert all(T == size(a, 4) for a in arrs)

      out = Array{Float64}(undef, H, W, C, T)
      curr = 1
      for (i, A) in enumerate(arrs)
        c = size(A, 3)
        r = curr:(curr+c-1)
        out[:, :, r, :] .= A
        curr += c
      end
      out
   end

cat3d_ (generic function with 1 method)

In [ ]:
function cat3d(arr1, arr2)
    R = size(arr1)[1]
    C = size(arr1)[2]
    Ch = size(arr1)[3]+size(arr2)[3]
    S = size(arr1)[4]
    
    arr1 = flatten(permutedims(arr1, [1,2,4,3]))
    arr2 = flatten(permutedims(arr2, [1,2,4,3]))
    out = reshape(cat1d(arr1,arr2),R,C,S,Ch)
    out = permutedims(out,[1,2,4,3])
    
    return out
end

In [ ]:
function predict(w,x_in)
    
    # TODO:
    # include batch norm: https://github.com/denizyuret/Knet.jl/blob/master/src/batchnorm.jl
    
    # contraction
    #block 1
    # 256x256x2 -> 256x256x64
    conv_1_1 = relu.(conv4(w[1],x_in, padding = 1) .+ w[2]) # bock 1 convolution 1
    # 256x256x64 -> 256x256x64 (conc with block 7)
    conv_1_2 = relu.(conv4(w[3],conv_1_1, padding = 1) .+ w[4]) # block 1 convolution 2, incl max pooling
    # 256-256-64 -> 128x128x64
    pool_1 = pool(conv_1_2)
    
    # block 2
    # 128x128x64 -> 128x128x128
    conv_2_1 = relu.(conv4(w[5],pool_1, padding = 1) .+ w[6])
    # 128x128x128 -> 128x128x128 (conc with block 6)
    conv_2_2 = relu.(conv4(w[7],conv_2_1, padding = 1) .+ w[8])
    # 128x128x128 -> 64x64x128
    pool_2 = pool(conv_2_2)
    
    #block 3
    # 64x64x128 -> 64x64x256
    conv_3_1 = relu.(conv4(w[9],pool_2, padding = 1) .+ w[10])
    #64x64x256 -> 64x64x256
    conv_3_2 = relu.(conv4(w[11],conv_3_1, padding = 1) .+ w[12])
    # 64x64x256 -> 32x32x256 (conc with block 5)
    pool_3 = pool(conv_3_2)
    
    # bottleneck 
    # block 4
    # 32x32x256 -> 32x32x512
    conv_4_1 = relu.(conv4(w[13], pool_3, padding = 1) .+ w[14])
    #32x32x512 -> 32x32x512
    conv_4_2 = relu.(conv4(w[15], conv_4_1, padding = 1) .+ w[16])
    
    # upsampling
    # block 5
    # upsampling/"deconvolution": 32x32x512 -> 64x64x512
    upsampling_kernel_5 = convert(KnetArray, bilinear(Float64,2,2,512,512))
    upsampling_5 = deconv4(upsampling_kernel_5, conv_4_2, padding = 1,stride = 2)
    # concatenation with conv_3_2: 64x64x512 -> 64x64x768
    # conc_5 = convert(KnetArray,cat(convert(Array, upsampling_5), convert(Array, conv_3_2), dims = 3))
    cat_5 = cat3d(upsampling_5,conv_3_2)
    # 64x64x768 -> 64x64x256
    conv_5_1 = relu.(conv4(w[17], cat_5, padding = 1) .+ w[18])
    # 64x64x256-> 64x64x256
    conv_5_2 = relu.(conv4(w[19], conv_5_1, padding = 1) .+ w[20])
    
    # block 6
    # 64x64x256 -> 128x128x256
    upsampling_kernel_6 = convert(KnetArray, bilinear(Float64,2,2,256,256))
    upsampling_6 = deconv4(upsampling_kernel_6, conv_5_2, padding = 1,stride = 2) 
    # 128x128x256 -> 128x128x384
    # conc_6 = convert(KnetArray,cat(convert(Array,upsampling_6), convert(Array,conv_2_2), dims = 3)) # with conv_2_2
    cat_6 = cat3d(upsamling_6, conv_2_2)
    #128x128x384 -> 128x128x128
    conv_6_1 = relu.(conv4(w[21], cat_6, padding = 1) .+ w[22])
    # 128x128x128 -> 128x128x128
    conv_6_2 = relu.(conv4(w[23], conv_6_1, padding = 1) .+ w[24])
    
    # block 7
    # 128x128x128 -> 256x256x128
    upsampling_kernel_7 = convert(KnetArray, bilinear(Float64,2,2,128,128))
    upsampling_7 = deconv4(upsampling_kernel_7, conv_6_2, padding = 1,stride = 2) 
    # 256x256x128 -> 256x256x192
    # conc_7 = convert(KnetArray,cat(convert(Array, upsampling_7), convert(Array, conv_1_2), dims = 3))
    cat_7 = cat3d(upsamling_7, conv_1_2)
    # 256x256x192 -> 256x256x64
    conv_7_1 = relu.(conv4(w[25], cat_7, padding = 1) .+ w[26])
    # 256x256x64 -> 256x256x64
    conv_7_2 = relu.(conv4(w[27], conv_7_1, padding = 1) .+ w[28])
    # 256x256x64 -> 256x256x2 (1x1 convolution)
    conv_7_3 = sigm.(conv4(w[29], conv_7_2).+ w[30])
    
    return conv_7_3
end
    

In [ ]:
function conv_layer_encoder(w, bn_moments, bn_params, x)
    out = conv4(w[1], x, padding=1).+ w[2]
    out = batchnorm(out, bn_moments[1], bn_params[1])
    conv_1 = relu.(out)
    
    out = conv4(w[3], conv_1, padding=1) .+ w[4]
    out = batchnorm(out, bn_moments[2], bn_params[2])
    out = relu.(out)
    pooled = pool(out)
    
    return conv_1, pooled
end

In [2]:
function conv_layer_decoder(w, bn_moments, bn_params, x_cat, x)
    number_of_channels = size(x)[3]
    upsampling_kernel = convert(KnetArray, bilinear(Float64,2,2,number_of_channels,number_of_channels))
    out = deconv4(upsampling_kernel, x, padding = 1,stride = 2) 
    out = cat3d(out, x_cat)
    
    out = conv4(w[1], out, padding = 1) .+ w[2]
    out = batchnorm(out, bn_moments[1], bn_params[1])
    out = relu.(out)
    
    out = conv4(w[3], out, padding = 1) .+ w[4]
    out = batchnorm(out, bn_moments[2], bn_params[2])
    out = relu.(out)
    
    return out
end

conv_layer_decoder (generic function with 1 method)

In [3]:
function bottleneck(w, bn_moments, bn_params, x)
    out = conv4(w[1], x, padding = 1) .+ w[2]
    out = batchnorm(out, bn_moments[1], bn_params[1])
    out = relu.(out)
    out = conv4(w[3], x, padding = 1) .+ w[4]
    out = batchnorm(out, bn_moments[2], bn_params[2])
    out = relu.(out)
    
    return out
end

bottleneck (generic function with 1 method)

In [ ]:
function output_layer(w, x)
    out = sigm.(conv4(w[1], x).+ w[2])
    
    return out
end

In [ ]:
function predict(w,bn_moments, bn_params,x_in)
    # layer 1
    conv_1, out = conv_layer_encoder(w[1:4], bn_moments[1:2], bn_params[1:2], x_in)
    # layer 2
    conv_2, out = conv_layer_encoder(w[5:8], bn_moments[3:4], bn_params[3:4], out)
    #layer 3
    conv_3, out = conv_layer_encoder(w[9:12], bn_moments[5:6], bn_params[5:6], out)
    # layer 4 = bottleneck
    # out = bottleneck(w[13:16], bn_moments[7:8], bn_params[7:8], out)
    out = conv4(w[13], out, padding = 1) .+ w[14]
    out = batchnorm(out, bn_moments[7], bn_params[7])
    out = relu.(out)
    out = conv4(w[15], x, padding = 1) .+ w[16]
    out = batchnorm(out, bn_moments[8], bn_params[8])
    out = relu.(out)
    # layer 5
    out = conv_layer_decoder(w[17:20], bn_moments[9:10], bn_params[9:10], conv_3, out)
    # layer 6
    out = conv_layer_decoder(w[21:24], bn_moments[11:12], bn_params[11:12], conv_2, out)
    # layer 7
    out = conv_layer_decoder(w[25:28], bn_moments[13:14], bn_params[13:14], conv_1, out)
    out = output_layer(w[29:30], out)
    
    return out
end

# loss
NOT IMPLEMENTED

In [ ]:
# dice loss
loss(w, x, ygold, predict) = nll(predict(w, x), ygold);
lossgradient = grad(loss);

In [ ]:
function flatten(array)
    size_array = size(array)
    length_flattened = 1
    for dim = 1:ndims(array)
        length_flattened = length_flattened * size_array[dim]
    end
    return flattened = reshape(array,length_flattened)
end

In [ ]:
# categorical cross-entropy
function categorical_crossentropy_3d(y_true, y_pred):
    epsilon = 1.0e-7
    y_true_flatten = flatten(y_true)
    y_pred_flatten = flatten(y_pred)
    y_pred_flatten_log = log.(y_pred_flatten.+epsilon)
    num_total_elements = sum(y_true_flatten) # number of positive voxels (gt 0/1 encoded)
    cross_entropy = sum(y_true_flatten.*y_pred_flatten_log)
    mean_cross_entropy = cross_entropy/(num_total_elements+epsilon)
    return mean_cross_entropy
end

# train
NOT IMPLEMENTED

In [ ]:
function train(w, dtrn, optim, predict; epochs=10)
    
    for epoch = 1:epochs
        for (x, y) in dtrn
            g = lossgradient(w, x, y, predict)
            update!(w, g, optim)
        end
    end
end

In [ ]:
optim(w; lr=0.1) = optimizers(w, Sgd;  lr=lr);

In [ ]:
function report(epoch, w, dtrn, dtst, predict)
    println((:epoch, epoch, :trn, accuracy(w, dtrn, predict), :tst, accuracy(w, dtst, predict)))
end

In [ ]:
w   = initweights(atype);
opt = optim(w);

if fast
    train(w, dtrn, opt, predict; epochs=nepochs)
else
    for epoch = 1:nepochs
        train(w, dtrn, opt, predict; epochs=1)
        report(epoch, w, dtrn, dtst, predict)
    end
end